In [3]:
import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "portfolio_management.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

from common.models import Brokers, Prices, Transactions, FX, Assets
from users.models import CustomUser
from utils import NAV_at_date, Irr, chart_dates, chart_labels, chart_colour, portfolio_at_date, calculate_security_nav, currency_format, calculate_portfolio_value
from datetime import date
from django.db.models import F, Sum

In [4]:
selected_brokers = broker_ids = [2]
effective_current_date = date.today()
currency_target = 'USD'
number_of_digits = 2

CustomUser.objects.get(id=2)

<CustomUser: Yaroslav>

In [5]:
security_id = 7
security = Assets.objects.get(id=security_id)
security.price_at_date(effective_current_date).price

quote = security.prices.filter(date__lte=effective_current_date).order_by('-date').first()
print(currency_target)
if currency_target is not None:
    quote.price = quote.price * FX.get_rate(security.currency, currency_target, effective_current_date)['FX']
quote

USD


<Prices: Long/short is at 24.00000000 on 2020-12-01>

In [9]:
import pandas as pd

from decimal import Decimal, ROUND_DOWN

def parse_excel_file_transactions(file, currency, broker_id):
    df = pd.read_excel(file, header=None)
    securities = []
    transactions = []
    i = 0
    
    quantity_field = Transactions._meta.get_field('quantity')
    quantity_decimal_places = quantity_field.decimal_places
    price_field = Transactions._meta.get_field('price')
    price_decimal_places = price_field.decimal_places
    
    while i < len(df.columns):
        if pd.notna(df.iloc[1, i]):
            security_name = df.iloc[1, i]
            isin = df.iloc[2, i]
            securities.append({'name': security_name, 'isin': isin, 'currency': currency})

            transactions_start = df[df.iloc[:, i] == 'Дата'].index[0] + 1

            for row in range(transactions_start, len(df)):
                if pd.isna(df.iloc[row, i]):
                    continue

                date = df.iloc[row, i].strftime("%Y-%m-%d")
                price = round(Decimal(df.iloc[row, i + 1]), price_decimal_places)
                quantity = round(Decimal(df.iloc[row, i + 2]), quantity_decimal_places) if not pd.isna(df.iloc[row, i + 2]) else None
                
                print("utils. 1047. Price: ", price, "Quantity: ", quantity)

                # Validate the quantity
                
                # # If the number of decimal places exceeds the allowed decimal places,
                # # round the quantity to the allowed number of decimal places
                # if quantity is not None:
                #     if int(quantity.as_tuple().exponent) < -decimal_places:
                #         quantity = quantity.quantize(Decimal(10) ** -decimal_places, rounding=ROUND_DOWN)

                dividend = round(Decimal(df.iloc[row, i + 3]), 2) if not pd.isna(df.iloc[row, i + 3]) else None
                commission = round(Decimal(df.iloc[row, i + 4]), 2) if not pd.isna(df.iloc[row, i + 4]) else None

                if quantity is None and dividend is None and commission is None:
                    print("utils. 1059. Skipping row for: ", security_name)
                    continue

                if quantity is not None:
                    if quantity > 0:
                        transaction_type = 'Buy'
                    elif quantity < 0:
                        transaction_type = 'Sell'
                    else:
                        transaction_type = 'Dividend'
                else:
                    transaction_type = None

                transaction_data = {
                    # 'broker': broker_id,
                    'security_name': security_name,
                    # 'isin': isin,
                    'date': date,
                    'type': transaction_type,
                    # 'currency': currency,
                    'price': price,
                    'quantity': quantity,
                    'transaction_value': round(price * quantity, 2) if quantity is not None else None,
                    # 'cash_flow': dividend,
                    # 'commission': commission,
                }
                transactions.append(transaction_data)
            i += 1
            while i < len(df.columns) and pd.isna(df.iloc[1, i]):
                i += 1
        else:
            i += 1

    return securities, transactions

file_path = "../Data/Test -- Transactions.xlsx"

securities, transactions = parse_excel_file_transactions(file_path, 'CUR', [0])
transactions

utils. 1047. Price:  1.217500 Quantity:  413.330500
utils. 1047. Price:  1.153600 Quantity:  182.038800
utils. 1047. Price:  0.888600 Quantity:  236.326900
utils. 1047. Price:  0.954100 Quantity:  220.102700
utils. 1047. Price:  0.985200 Quantity:  213.154700
utils. 1047. Price:  0.954100 Quantity:  220.102700
utils. 1047. Price:  0.888100 Quantity:  236.459800
utils. 1047. Price:  0.918400 Quantity:  228.658500
utils. 1047. Price:  0.990099 Quantity:  212.099900
utils. 1047. Price:  1.019000 Quantity:  206.084400
utils. 1047. Price:  1.037000 Quantity:  166.297000
utils. 1047. Price:  1.001000 Quantity:  209.790200
utils. 1047. Price:  1.078600 Quantity:  270.415400
utils. 1047. Price:  1.162300 Quantity:  250.942100
utils. 1047. Price:  1.230200 Quantity:  237.091600
utils. 1047. Price:  1.238500 Quantity:  235.502600
utils. 1047. Price:  1.252700 Quantity:  232.833100
utils. 1047. Price:  1.233400 Quantity:  236.476400
utils. 1047. Price:  1.252100 Quantity:  232.944700
utils. 1047.

[{'security_name': 'UBS 60/40 A Pas',
  'date': '2008-08-25',
  'type': 'Buy',
  'price': Decimal('1.217500'),
  'quantity': Decimal('413.330500'),
  'transaction_value': Decimal('503.23')},
 {'security_name': 'UBS 60/40 A Pas',
  'date': '2008-09-25',
  'type': 'Buy',
  'price': Decimal('1.153600'),
  'quantity': Decimal('182.038800'),
  'transaction_value': Decimal('210.00')},
 {'security_name': 'UBS 60/40 A Pas',
  'date': '2008-10-25',
  'type': 'Buy',
  'price': Decimal('0.888600'),
  'quantity': Decimal('236.326900'),
  'transaction_value': Decimal('210.00')},
 {'security_name': 'UBS 60/40 A Pas',
  'date': '2008-11-25',
  'type': 'Buy',
  'price': Decimal('0.954100'),
  'quantity': Decimal('220.102700'),
  'transaction_value': Decimal('210.00')},
 {'security_name': 'UBS 60/40 A Pas',
  'date': '2008-12-25',
  'type': 'Buy',
  'price': Decimal('0.985200'),
  'quantity': Decimal('213.154700'),
  'transaction_value': Decimal('210.00')},
 {'security_name': 'UBS 60/40 A Pas',
  'date